<a href="https://colab.research.google.com/github/venidaydream/Intro_to_Big_Data/blob/main/Venelina_tracking_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio

Considerando los stocks de IBEX que ya trabajaste, crear una cartera equiprobable con los stocks que consideraste para tu carteara (en el ejercicio que no he calificado) y comparar con el IBEX a partir del Tracking Error.

El tracking error mide la diferencia entre el rendimiento de una cartera y el rendimiento de su índice de referencia. Puede indicar cuán bien la cartera sigue o se desvía del índice.

Debes concluir si tu cartera sigue el IBEX o no.

Y finalmente hacer la función donde dada la distribución de los retornos de la cartera y la distribución de los retornos del Índice, en entregue el tracking error.

# Inicializar la cartera y el IBEX35

De la entrega de IBEX35, mi cartera final tenía las siguientes acciones: ACCIONA, AENA y SOLARIA ENERGIA

In [269]:
import pandas as pd
import datetime
import yfinance as yf
import numpy as np

In [270]:
#ACCIONA, AENA y SOLARIA ENERGIA - los ultimos 3 años
start_date = "2020-01-01"
end_date = "2023-11-05"
symbols = ['ANA.MC','SLR.MC','AENA.MC']
cartera = yf.download(symbols, start=start_date, end=end_date)

cartera_adj_close = cartera['Adj Close']
cartera_adj_close.head().round(4)

[*********************100%%**********************]  3 of 3 completed


,AENA.MC,ANA.MC,SLR.MC
Date,,,
2020-01-02,155.0192,83.2112,7.15
2020-01-03,153.9332,81.9518,7.06
2020-01-06,152.9378,81.1872,6.84
2020-01-07,152.2591,81.1872,6.94
2020-01-08,152.5758,81.1422,6.95


In [271]:
ibex35 = yf.download('^IBEX', start=start_date, end=end_date)
ibex35_adj_close = ibex35['Adj Close']
ibex35_adj_close.head().round(4)

[*********************100%%**********************]  1 of 1 completed


Date
2020-01-02    9691.2002
2020-01-03    9646.5996
2020-01-06    9600.9004
2020-01-07    9579.7998
2020-01-08    9591.4004
Name: Adj Close, dtype: float64

# Retornos

In [272]:
#retornos simples diarios de la cartera
returns_cartera = cartera_adj_close.pct_change().dropna()
print(returns_cartera.round(4))

            AENA.MC  ANA.MC  SLR.MC
Date                               
2020-01-03  -0.0070 -0.0151 -0.0126
2020-01-06  -0.0065 -0.0093 -0.0312
2020-01-07  -0.0044  0.0000  0.0146
2020-01-08   0.0021 -0.0006  0.0014
2020-01-09   0.0083  0.0200  0.0676
...             ...     ...     ...
2023-10-30   0.0097  0.0144  0.0004
2023-10-31   0.0077  0.0245  0.0220
2023-11-01   0.0106 -0.0063 -0.0049
2023-11-02   0.0546  0.0469  0.0391
2023-11-03  -0.0031  0.0275  0.0383

[986 rows x 3 columns]


In [273]:
#retornos simples diarios de IBEX35
returns_ibex = ibex35_adj_close.pct_change().dropna()
print(returns_ibex.round(4))

Date
2020-01-03   -0.0046
2020-01-06   -0.0047
2020-01-07   -0.0022
2020-01-08    0.0012
2020-01-09   -0.0010
               ...  
2023-10-30    0.0107
2023-10-31    0.0004
2023-11-01    0.0064
2023-11-02    0.0204
2023-11-03    0.0036
Name: Adj Close, Length: 986, dtype: float64


# Tracking error

Formulas de: https://www.investopedia.com/terms/t/trackingerror.asp#:~:text=Given%20a%20sequence%20of%20returnsand%20B%20is%20benchmark%20return.



Version 1
- diferencia entre el retorno de la cartera y el retorno de IBEX35 simple
- cada activo tiene el mismo peso: 1/3 de la cartera
- hecha con retornos diarios

In [274]:
weighted_portfolio_return = (returns_cartera['AENA.MC']+returns_cartera['ANA.MC']+returns_cartera['SLR.MC'])/3
tracking_error1 = weighted_portfolio_return-returns_ibex
tracking_error1*100 #%

Date
2020-01-03   -0.697375
2020-01-06   -1.091556
2020-01-07    0.559179
2020-01-08   -0.022191
2020-01-09    3.296265
                ...   
2023-10-30   -0.257146
2023-10-31    1.772436
2023-11-01   -0.661424
2023-11-02    2.644620
2023-11-03    1.726699
Length: 986, dtype: float64

Version 2
- Desviación Estándar de la diferencia entre el retorno de cartera y retorno de IBEX35

In [275]:
tracking_error1 #la diferencia

Date
2020-01-03   -0.006974
2020-01-06   -0.010916
2020-01-07    0.005592
2020-01-08   -0.000222
2020-01-09    0.032963
                ...   
2023-10-30   -0.002571
2023-10-31    0.017724
2023-11-01   -0.006614
2023-11-02    0.026446
2023-11-03    0.017267
Length: 986, dtype: float64

In [276]:
tracking_errors2 = tracking_error1.std() * (252 ** 0.5) #anualizar la desviacion de los retornos diarios
round(tracking_errors2*100,2) #%

21.92

# La función del tracking error

In [277]:
import pandas as pd
import datetime
import yfinance as yf
import numpy as np

def tracking_error(symbols, index, start_date, end_date):

  cartera = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
  index = yf.download(index, start=start_date, end=end_date)['Adj Close']

  cartera_returns = cartera.pct_change().dropna()
  index_returns = index.pct_change().dropna()

  weighted_portfolio_return = cartera_returns.mean(axis=1)
  tracking_error_2 = (weighted_portfolio_return - index_returns).std() * (252 ** 0.5)

  print("Tracking Error:", f"{(tracking_error_2*100):.2f}%")


start_date = "2020-01-01"
end_date = "2023-11-05"
symbols = ['ANA.MC','SLR.MC','AENA.MC']
index = '^IBEX'
tracking_error = tracking_error(symbols, index, start_date, end_date)

[*********************100%%**********************]  3 of 3 completed
[*********************100%%**********************]  1 of 1 completed
Tracking Error: 21.92%


# Conclusion

El resultado de tracking error = 21,92% comparando mi cartera de ACCIONA, AENA y SOLARIA ENERGIA de los ultimos 3 años con IBEX35 significa en promedio los rendimientos de mi cartera desvían 22% de los rendimientos del indice IBEX35. Por eso, diría que unos 22% son bastante alto porcentaje de desviación y que mi cartera no sigue el IBEX35 en los ultimos 3 años.